# Example Sample Hotel and Flight Booker Agent 

Denna lösning hjälper dig att boka flygbiljetter och hotell. Scenariot är en resa från London Heathrow LHR 20 februari 2024 till New York JFK med retur 27 februari 2025, ekonomi med British Airways endast. Jag vill bo på ett Hilton-hotell i New York, vänligen ange kostnader för flyg och hotell.


# Initiera Azure AI Agent-tjänsten och hämta konfigurationsinformation från **.env**

### **.env** 

Skapa en .env-fil 

**.env** innehåller anslutningssträngen för Azure AI Agent-tjänsten, modellen som används av AOAI, och motsvarande Google API Search-tjänstens API, ENDPOINT, etc.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "Ditt Azure AI Agent Service Model Deployment-namn"

[**NOTE**] Du behöver en modell med 100 000 Rate Limit (Tokens per minut) och Rate Limit på 600 (Request per minut)

  Du kan få modellen i Microsoft Foundry - Model and Endpoint.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "Din Azure AI Agent Service Project Connection String"

  Du kan få projektets anslutningssträng i din projektöversikt i AI Foundry Portal-skärmen.

- **SERPAPI_SEARCH_API_KEY** = "Din SERPAPI Search API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "Din SERPAPI Search Endpoint"

För att få Model Deployment Name och Project Connection String för Azure AI Agent Service behöver du skapa Azure AI Agent Service. Det rekommenderas att använda [denna mall](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) för att skapa den direkt（***Note:*** Azure AI Agent Service är för närvarande satt i en begränsad region. Det rekommenderas att du hänvisar till [denna länk](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) för att ställa in regionen）

Agent behöver åtkomst till SERPAPI. Det rekommenderas att registrera sig via [denna länk](https://serpapi.com/searches). Efter registrering kan du få en unik API KEY och ENDPOINT.


# Setup 

För att köra denna anteckningsbok måste du se till att du har installerat de nödvändiga biblioteken genom att köra `pip install -r requirements.txt`.


In [ ]:
from semantic_kernel import __version__

__version__

Din Semantic Kernel-version bör vara minst 1.27.2.


Ladda dina inställningar och resurser från .env-filen, vänligen säkerställ att du har lagt till dina nycklar och inställningar samt skapat en lokal .env-fil.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Logga in på Azure

Du behöver nu logga in på Azure. Öppna en terminal och kör följande kommando:

```bash
az login
```

Det här kommandot kommer att be dig ange dina Azure-uppgifter, vilket möjliggör att Azure AI Agent-tjänsten fungerar korrekt.


# Förklaring:
Detta är en variabel som lagrar API-nyckeln för att få åtkomst till en SERP (Search Engine Results Page) API-tjänst. En API-nyckel är en unik identifierare som används för att autentisera förfrågningar kopplade till ditt konto.

Syfte: Syftet med denna rad är att lagra API-nyckeln i en variabel så att den kan användas för att autentisera förfrågningar till SERP API-tjänsten. API-nyckeln krävs för att få åtkomst till tjänsten och utföra sökningar.
Hur du får en SERP API-nyckel: För att få en SERP API-nyckel, följ dessa allmänna steg på https://serpapi.com (de exakta stegen kan variera beroende på vilken specifik SERP API-tjänst du använder):

Välj en SERP API-tjänst: Det finns flera SERP API-tjänster tillgängliga, såsom SerpAPI, Google Custom Search JSON API och andra. Välj den som bäst passar dina behov.

Registrera ett konto: Gå till webbplatsen för den valda SERP API-tjänsten och registrera ett konto. Du kan behöva lämna viss grundläggande information och bekräfta din e-postadress.

Skapa en API-nyckel: Efter registrering, logga in på ditt konto och navigera till API-sektionen eller instrumentpanelen. Leta efter ett alternativ för att skapa eller generera en ny API-nyckel.
Kopiera API-nyckeln till din .env-fil.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# Explanation:
BASE_URL: Detta är en variabel som lagrar bas-URL:en för SERP API-slutpunkten. Variabelnamnet BASE_URL är en konvention som används för att indikera att denna URL är startpunkten för att göra API-förfrågningar.
'https://serpapi.com/search':

Detta är själva URL-strängen som tilldelas variabeln BASE_URL. Den representerar slutpunkten för att utföra sökfrågor med hjälp av SERP API.

# Purpose:
Syftet med denna rad är att definiera en konstant som håller bas-URL:en för SERP API. Denna URL kommer att användas som startpunkt för att konstruera API-förfrågningar för att utföra sökoperationer.

# Usage:
Genom att definiera bas-URL:en i en variabel kan du enkelt återanvända den i din kod när du behöver göra förfrågningar till SERP API. Detta gör din kod mer underhållbar och minskar risken för fel från att hårdkoda URL:en på flera ställen. Det aktuella exemplet är https://serpapi.com/search?engine=bing som använder Bing-söknings-API. Olika API:er kan väljas på https://Serpapi.com


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# Förklaring:

Här är din plugin-kod placerad.

Klassdefinition: `class BookingPlugin`: Definierar en klass med namnet BookingPlugin som innehåller metoder för bokning av hotell och flyg.

Metod för hotellbokning:

- `@kernel_function(description="booking hotel")`: En dekoratör som beskriver funktionen som en kernel-funktion för hotellbokning.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: Definierar en metod för hotellbokning med annoterade parametrar och returtyp.

Metoden konstruerar en ordbok med parametrar för hotellbokningsförfrågan och skickar en GET-förfrågan till SERP API. Den kontrollerar svarstatus och returnerar hotellens egenskaper om det lyckas, eller None om förfrågan misslyckades.

Metod för flygbokning:

- `@kernel_function(description="booking flight")`: En dekoratör som beskriver funktionen som en kernel-funktion för flygbokning.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: Definierar en metod för flygbokning med annoterade parametrar och returtyp.

Metoden konstruerar ordböcker med parametrar för utresan och returflygförfrågningar och skickar GET-förfrågningar till SERP API. Den kontrollerar svarstatus och lägger till flyginformationen i resultatsträngen om det lyckas, eller skriver ut ett felmeddelande om förfrågan misslyckades. Metoden returnerar resultatsträngen som innehåller flyginformationen.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# Förklaring:
Importsatser: Importerar nödvändiga moduler för Azure-referenser, AI-agent, chattmeddelandeinnehåll, författarroll och kernel-funktionsdekoration.

Asynkront kontextadministratör: async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,): Detta upprättar en asynkron kontextadministratör för att hantera Azure-referenser och skapa en AI-agentklient.

Agentnamn och instruktioner: 
- `AGENT_NAME = "BookingAgent"`: Definierar agentens namn.
- `AGENT_INSTRUCTIONS = """..."""`: Ger detaljerade instruktioner till agenten om hur bokningsförfrågningar ska hanteras.

Skapa agentdefinition: `agent_definition = await client.agents.create_agent(...)`: Skapar en agentdefinition med den angivna modellen, namnet och instruktionerna.

Skapa AzureAI-agent: `agent = AzureAIAgent(...)`: Skapar en AzureAI-agent med hjälp av klienten, agentdefinitionen och den definierade pluginen.

Skapa tråd: `thread: AzureAIAgentThread | None = None`: Skapar en tråd för agenten. Det är inte nödvändigt att först skapa en tråd – om värdet `None` anges kommer en ny tråd att skapas vid första anropet och returneras som en del av svaret.

Användarinmatningar: `user_inputs = ["..."]`: Definierar en lista med användarinmatningar som agenten ska bearbeta.

I finally-blocket tas tråden och agenten bort för att frigöra resurser.


# Autentisering

Klassen `DefaultAzureCredential` är en del av Azure SDK för Python. Den tillhandahåller ett standard sätt att autentisera mot Azure-tjänster. Den försöker autentisera med flera metoder i en specifik ordning, såsom miljövariabler, hanterad identitet och Azure CLI-referenser.

Asynkrona operationer: aio-modulen indikerar att klassen DefaultAzureCredential stödjer asynkrona operationer. Det innebär att du kan använda den med asyncio för att utföra icke-blockerande autentiseringsförfrågningar.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Ansvarsfriskrivning**:
Detta dokument har översatts med hjälp av AI-översättningstjänsten [Co-op Translator](https://github.com/Azure/co-op-translator). Även om vi strävar efter noggrannhet bör du vara medveten om att automatiska översättningar kan innehålla fel eller felaktigheter. Det ursprungliga dokumentet på dess modersmål bör betraktas som den auktoritativa källan. För viktig information rekommenderas professionell mänsklig översättning. Vi ansvarar inte för eventuella missförstånd eller feltolkningar som uppstår till följd av användningen av denna översättning.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
